# Correlation matrix and visualization of variables and GDP

### This is the master notebook, please copy and create a new one for each group.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro

In [78]:
df= pd.read_csv (os.getcwd()+'/Data/'+'Persian Gulf.csv')
df

,Unnamed: 0,Country,Year,Alcohol per capita,Education GExp,Employment-agriculture,Employment-industry,Employment-services,Exports-Commercial services,Exports-G&S,...,International taxes,Literacy rate,Mortality-infants,Net migration,Ninis,R&D GExp,Renewable electricity,Suicide,Workers high education,Continent
0,692,IRQ,1990,0.666667,NaN,1.000000,0.001855,0.018328,0.000000,1.431296e-01,...,0.0,0.104822,0.487480,0.367585,1.0,0.992902,NaN,0.333333,1.0,Persian Gulf
1,693,IRQ,1991,0.666667,NaN,1.000000,0.001855,0.018328,0.000000,4.708877e-05,...,0.0,0.104822,0.510955,0.367585,1.0,0.992902,NaN,0.333333,1.0,Persian Gulf
2,694,IRQ,1992,0.666667,NaN,0.990957,0.014842,0.024055,0.000000,2.608861e-05,...,0.0,0.104822,0.547209,0.367585,1.0,0.992902,NaN,0.333333,1.0,Persian Gulf
3,695,IRQ,1993,0.666667,NaN,0.978146,0.009277,0.046964,0.000000,8.898371e-07,...,0.0,0.104822,0.601461,0.456732,1.0,0.992902,NaN,0.333333,1.0,Persian Gulf
4,696,IRQ,1994,0.666667,NaN,0.963828,0.001855,0.072165,0.000000,7.485220e-06,...,0.0,0.104822,0.670579,0.545878,1.0,0.992902,NaN,0.333333,1.0,Persian Gulf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,1031,OMN,2016,0.052209,0.906716,0.081218,0.852442,0.185883,0.721899,4.799576e-01,...,NaN,0.947401,0.277970,0.667308,NaN,1.000000,NaN,0.000000,NaN,Persian Gulf
214,1032,OMN,2017,0.056225,1.000000,0.055838,0.839213,0.205839,0.808696,5.886064e-01,...,NaN,0.994714,0.298731,0.584135,NaN,0.645847,NaN,0.000000,NaN,Persian Gulf
215,1033,OMN,2018,0.060241,0.871258,0.025381,0.850746,0.200296,0.937194,7.603027e-01,...,NaN,1.000000,0.307958,0.584135,NaN,0.551994,NaN,0.000000,NaN,Persian Gulf
216,1034,OMN,2019,0.060241,0.829429,0.000000,0.834803,0.222469,1.000000,7.100584e-01,...,NaN,1.000000,0.305652,0.584135,NaN,0.551994,NaN,0.095238,NaN,Persian Gulf


## Visualization

In [79]:
columns=['Country','Year','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']
clist=df['Country'].unique()

The following loop has been created to test if each variable for each country follows a normal distribution and also to present in an orderly manner all the correlations between the GDP and the variables so they can be understood on a lower scale.

In [80]:
def multcolumn(frame):
    for u in range(3, len(columns)):
        name2=columns[u]+'.^2'
        name3=columns[u]+'.^3'
        namelog=columns[u]+'.log'
        frame.loc[:,name2] = frame[columns[u]]**2
        frame.loc[:,name3] = frame[columns[u]]**3
        frame.loc[:,namelog] = np.log(frame[columns[u]])

In [81]:
multcolumn(df)
for i in range(0,len(clist)):
    dat=df.loc[df.loc[:, 'Country'] == clist[i]]
    for e in range(3,len(columns)):
        data=dat.iloc[:, e]
        stat, p = shapiro(data)
        print(clist[i] +"-"+ columns[e])
        print('Statistical=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            print('Data is NORMAL ( H0 not denied )')
        else:
            print('It does NOT seem normal( Denies H0 )')
    print(clist[i])
    cor=dat.corr()    
    cor.loc[:,'GDP-R^2'] = cor['GDP']**2
    cor.loc[:,'Indicator']=cor.index
    cor[['Indicator','Type']]=cor.Indicator.str.split('.',expand=True)  
    corcolumn=cor[['Indicator','Type','GDP-R^2','GDP']]
    corcolumn=corcolumn.loc[corcolumn.loc[:, 'GDP-R^2'] >= 0.75]
    id=corcolumn.groupby('Indicator')['GDP-R^2'].transform(max)==corcolumn['GDP-R^2']
    corcolumn[id]
    max_df=pd.DataFrame(corcolumn[id])
    max_df['Behaviour']=np.where(max_df['GDP']>0, 'Positive', 'Negative')
    max_df['Type']=max_df['Type'].replace(['^2','^3','log'],['Cuadratic','Cubic','Logarithmic'])
    max_df.drop("GDP",axis=1,inplace=True)
    max_df=max_df.reset_index(drop=True)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Year'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='GDP'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Unnamed: 0'].index)
    max_df=max_df.sort_values(by = 'GDP-R^2',ascending = False).style.background_gradient()
    display(max_df)

IRQ-Renewable electricity
Statistical=0.838, p=0.000
It does NOT seem normal( Denies H0 )
IRQ-Employment-agriculture
Statistical=nan, p=1.000
Data is NORMAL ( H0 not denied )
IRQ-Employment-industry
Statistical=0.916, p=0.019
It does NOT seem normal( Denies H0 )
IRQ-Employment-services
Statistical=0.904, p=0.009
It does NOT seem normal( Denies H0 )
IRQ-Exports-G&S
Statistical=0.904, p=0.009
It does NOT seem normal( Denies H0 )
IRQ-Fertility rate
Statistical=0.741, p=0.000
It does NOT seem normal( Denies H0 )
IRQ-Foreign investment
Statistical=0.901, p=0.007
It does NOT seem normal( Denies H0 )
IRQ-GDP
Statistical=0.965, p=0.390
Data is NORMAL ( H0 not denied )
IRQ-Education GExp
Statistical=0.744, p=0.000
It does NOT seem normal( Denies H0 )
IRQ-Workers high education
Statistical=0.869, p=0.001
It does NOT seem normal( Denies H0 )
IRQ-Literacy rate
Statistical=0.815, p=0.000
It does NOT seem normal( Denies H0 )
IRQ-Net migration
Statistical=0.442, p=0.000
It does NOT seem normal( Denie

c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zer

,Indicator,Type,GDP-R^2,Behaviour
0,Exports-G&S,None,0.834043,Positive
3,Workers high education,Cuadratic,0.802727,Negative
5,Ninis,Cuadratic,0.802727,Negative
4,Health services use,Logarithmic,0.784696,Positive
2,Employment-industry,Cuadratic,0.772082,Positive


QAT-Renewable electricity
Statistical=0.719, p=0.000
It does NOT seem normal( Denies H0 )
QAT-Employment-agriculture
Statistical=0.876, p=0.002
It does NOT seem normal( Denies H0 )
QAT-Employment-industry
Statistical=0.873, p=0.002
It does NOT seem normal( Denies H0 )
QAT-Employment-services
Statistical=0.860, p=0.001
It does NOT seem normal( Denies H0 )
QAT-Exports-G&S
Statistical=0.858, p=0.001
It does NOT seem normal( Denies H0 )
QAT-Fertility rate
Statistical=0.616, p=0.000
It does NOT seem normal( Denies H0 )
QAT-Foreign investment
Statistical=0.844, p=0.000
It does NOT seem normal( Denies H0 )
QAT-GDP
Statistical=0.879, p=0.002
It does NOT seem normal( Denies H0 )
QAT-Education GExp
Statistical=0.581, p=0.000
It does NOT seem normal( Denies H0 )
QAT-Workers high education
Statistical=0.823, p=0.000
It does NOT seem normal( Denies H0 )
QAT-Literacy rate
Statistical=nan, p=1.000
Data is NORMAL ( H0 not denied )
QAT-Net migration
Statistical=nan, p=1.000
Data is NORMAL ( H0 not deni

,Indicator,Type,GDP-R^2,Behaviour
3,Exports-G&S,None,0.952483,Positive
7,Fertility rate,Logarithmic,0.875687,Negative
5,Employment-agriculture,Logarithmic,0.863212,Negative
6,Employment-services,Cuadratic,0.792880,Negative
2,Employment-industry,None,0.790247,Positive


...

,Indicator,Type,GDP-R^2,Behaviour
1,Exports-G&S,None,0.944816,Positive
4,Education GExp,Logarithmic,0.885706,Negative
3,Health services use,None,0.800257,Positive
0,Exports-Commercial services,None,0.796173,Positive


YEM-Renewable electricity
Statistical=0.740, p=0.000
It does NOT seem normal( Denies H0 )
YEM-Employment-agriculture
Statistical=0.730, p=0.000
It does NOT seem normal( Denies H0 )
YEM-Employment-industry
Statistical=0.820, p=0.000
It does NOT seem normal( Denies H0 )
YEM-Employment-services
Statistical=0.942, p=0.091
Data is NORMAL ( H0 not denied )
YEM-Exports-G&S
Statistical=0.880, p=0.002
It does NOT seem normal( Denies H0 )
YEM-Fertility rate
Statistical=0.615, p=0.000
It does NOT seem normal( Denies H0 )
YEM-Foreign investment
Statistical=0.529, p=0.000
It does NOT seem normal( Denies H0 )
YEM-GDP
Statistical=0.931, p=0.046
It does NOT seem normal( Denies H0 )
YEM-Education GExp
Statistical=0.707, p=0.000
It does NOT seem normal( Denies H0 )
YEM-Workers high education
Statistical=0.900, p=0.007
It does NOT seem normal( Denies H0 )
YEM-Literacy rate
Statistical=0.857, p=0.001
It does NOT seem normal( Denies H0 )
YEM-Net migration
Statistical=nan, p=1.000
Data is NORMAL ( H0 not de

,Indicator,Type,GDP-R^2,Behaviour
2,Education GExp,Logarithmic,0.864661,Negative
1,Suicide,None,0.814694,Negative
3,Alcohol per capita,Cubic,0.803707,Negative


OMN-Renewable electricity
Statistical=0.706, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Employment-agriculture
Statistical=0.945, p=0.116
Data is NORMAL ( H0 not denied )
OMN-Employment-industry
Statistical=0.935, p=0.062
Data is NORMAL ( H0 not denied )
OMN-Employment-services
Statistical=0.840, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Exports-G&S
Statistical=0.846, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Fertility rate
Statistical=0.882, p=0.003
It does NOT seem normal( Denies H0 )
OMN-Foreign investment
Statistical=0.880, p=0.002
It does NOT seem normal( Denies H0 )
OMN-GDP
Statistical=0.733, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Education GExp
Statistical=0.820, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Workers high education
Statistical=0.828, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Literacy rate
Statistical=0.798, p=0.000
It does NOT seem normal( Denies H0 )
OMN-Net migration
Statistical=nan, p=1.000
Data is NORMAL ( H0 not denied

,Indicator,Type,GDP-R^2,Behaviour
8,Health services use,None,0.960696,Positive
6,Exports-G&S,None,0.938222,Positive
2,Alcohol per capita,None,0.935661,Negative
3,Employment-industry,None,0.920114,Positive
5,Exports-Commercial services,None,0.891151,Positive
4,Employment-services,None,0.883528,Negative
9,Literacy rate,None,0.875279,Positive
10,Net migration,None,0.828027,Positive
